In [ ]:
import pandas as pd
import numpy as np
import pickle
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

df = pd.read_csv("Mental_Health_FAQ.csv",on_bad_lines='skip')
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [ ]:
questions = df["Questions"].values
answers = df["Answers"].values

In [ ]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(questions)

In [ ]:
with open("label_encoder.pickle", "wb") as enc_file:
    pickle.dump(label_encoder, enc_file)

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(questions)

In [ ]:
with open("tokenizer.pickle", "wb") as tok_file:
    pickle.dump(tokenizer, tok_file)

# Prepare training data
sequences = tokenizer.texts_to_sequences(questions)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=20)

In [ ]:
model = Sequential([
    Embedding(input_dim=2000, output_dim=16, input_length=20),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(labels), epochs=100)

# Save model
model.save("chat_model.keras")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0093 - loss: 4.5854   
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0093 - loss: 4.5846     
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0434 - loss: 4.5844 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0247 - loss: 4.5841    
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0372 - loss: 4.5842
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0341 - loss: 4.5838 
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0289 - loss: 4.5828
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0248 - loss: 4.5826 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0248 - loss: 4.5822 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0155 - loss: 4.5818    
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0311 - loss: 4.5809 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0217 - l

In [ ]:
import numpy as np
import pickle
import random
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

def chat():
    model = keras.models.load_model('chat_model.keras')

    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    with open('label_encoder.pickle', 'rb') as enc:
        label_encoder = pickle.load(enc)

    df = pd.read_csv('Mental_Health_FAQ.csv')
    max_len = 20

    print("Start messaging with the bot (type 'quit' to stop)!")

    while True:
        inp = input("User: ")
        if inp.lower() == "quit":
            break

        seq = tokenizer.texts_to_sequences([inp])
        padded = keras.preprocessing.sequence.pad_sequences(seq, truncating='post', maxlen=max_len)

        prediction = model.predict(padded)[0]
        tag_question = label_encoder.inverse_transform([np.argmax(prediction)])[0]

        # Find the corresponding answer
        answers = df[df['Questions'] == tag_question]['Answers'].values
        if len(answers) > 0:
            print("ChatBot:", random.choice(answers))
        else:
            print("ChatBot: Sorry, I don't understand.")

chat()


Start messaging with the bot (type 'quit' to stop)!
User: If I become involved in treatment, what do I need to know?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
ChatBot: Beginning treatment is a big step for individuals and families and can be very overwhelming. It is important to continue involvement in the treatment process as much as possible. Some questions you will need to have answered include:
What is known about the cause of this particular illness?
Are there other diagnoses where these symptoms are common?
Do you normally include a physical or neurological examination?
Are there any additional tests or exams that you would recommend at this point?
Would you advise an independent opinion from another psychiatrist at this point?
What program of treatment is the most helpful with this diagnosis?
Will this program involve services by other specialists? If so, who will be responsible for coordinating these services?
What do you see as the familyâ€™s role in this program of treatment?
How